In [1]:
import time
import json
import redis
import asyncio
import numpy as np
from aredis import StrictRedis

In [2]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [ ]:
date = time.strftime('%Y%m%d')

check_points_length = int(rd.get(f'hq_{date}_check_points_length'))


In [3]:
assist_count = int(rd.get('hq_assist_count'))
assist_count

8

In [ ]:
await ar.pubsub_channels()

In [ ]:
status = np.zeros(check_points_length, dtype=int)
status

In [ ]:
def message_handler(message):
    
    assist_idx = str(message['channel']).split('_')[2]
    data = json.loads(message['data'])
    
    if data['status'] == 'successful':
        check_point_idx = int(data['idx'])
        check_point = data['check_point']
        status[check_point_idx] += 1
        
        if status[check_point_idx] == assist_count:
            pass
        
        print('handler:', assist_idx, status[check_point_idx], message['data'])

In [ ]:
p = ar.pubsub()

await p.psubscribe(**{'hq_assist_*_snapshotting': message_handler})

while status[-1] != assist_count:
    
    message = await p.get_message()
        
    if np.sum(status) >= assist_count:
        break

await p.punsubscribe('hq_assist_*_snapshotting')
p.close()

In [ ]:
message

In [ ]:
status

In [16]:
results = await asyncio.gather(*[ ar.brpop(f'hq_assist_{_}_compute_statistics') for _ in range(assist_count)])


True